## objective detection

In [1]:
!pip install pydicom

In [2]:
import os
import time
import numpy as np
import pandas as pd
import pydicom as dicom

import torch
import torch.optim as optim
import torchvision.transforms as T
# from torchvision.utils import make_grid
from torch.utils.data import DataLoader, Dataset, random_split

from retinanet import model
from retinanet.dataloader import collater, Resizer, Augmenter, Normalizer, UnNormalizer

# import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

from tqdm.notebook import tqdm

In [3]:
from retinanet.csv_eval import _compute_ap, compute_overlap

In [4]:
df_annot = pd.read_csv("../rsna-pneumonia-detection-challenge/stage_2_train_labels.csv")
# df_annot = df_annot[df_annot['Target']==1]
df_annot

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1
...,...,...,...,...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,185.0,298.0,228.0,379.0,1
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,NaN,NaN,NaN,NaN,0
30224,c1f6b555-2eb1-4231-98f6-50a963976431,NaN,NaN,NaN,NaN,0
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,570.0,393.0,261.0,345.0,1


In [5]:
# def fmt(x):
#     if np.isnan(x):
#         return -1
#     else:
#         return x
    
# df_annot['x'] = df_annot['x'].apply(lambda x: fmt(x))
# df_annot['y'] = df_annot['y'].apply(lambda x: fmt(x))
# df_annot['width'] = df_annot['width'].apply(lambda x: fmt(x))
# df_annot['height'] = df_annot['height'].apply(lambda x: fmt(x))

In [6]:
unq_values = df_annot["patientId"].unique()
print("Total Records: ", len(df_annot))
print("Unique Images: ",len(unq_values))

null_values = df_annot.isnull().sum(axis = 0)
print("\n> Null Values in each column <")
print(null_values)

sources = df_annot["Target"].unique()
print("Total Target: ",len(sources))
print("\n> Target <\n",sources)

Total Records:  30227
Unique Images:  26684

> Null Values in each column <
patientId        0
x            20672
y            20672
width        20672
height       20672
Target           0
dtype: int64
Total Target:  2

> Target <
 [0 1]


In [7]:
root_path = '../rsna-pneumonia-detection-challenge/stage_2_train_images/'
train_data = []
for fp in os.listdir(root_path):
#     ds = dicom.dcmread(root_path+fp)
    train_data.append([fp.split('.')[0], root_path+fp])
    # plt.imshow(ds.pixel_array, cmap='gray')
#     break

In [8]:
df_data = pd.DataFrame(train_data, columns=['patientId','filepath']).sort_values(by='patientId')
# df_data = df_data.loc[df_data['patientId'].isin(df_annot['patientId'].unique())]
msk = np.random.rand(len(df_data)) < 0.8
df_train_data = df_data[msk]
df_valid_data = df_data[~msk]

In [9]:
df_train_data

,patientId,filepath
8888,000924cf-0f8d-42bd-9158-1af53881a557,../rsna-pneumonia-detection-challenge/stage_2_...
23483,000db696-cf54-4385-b10b-6b16fbb3f985,../rsna-pneumonia-detection-challenge/stage_2_...
13569,001031d9-f904-4a23-b3e5-2c088acd19c6,../rsna-pneumonia-detection-challenge/stage_2_...
8690,0010f549-b242-4e94-87a8-57d79de215fc,../rsna-pneumonia-detection-challenge/stage_2_...
17775,001916b8-3d30-4935-a5d1-8eaddb1646cd,../rsna-pneumonia-detection-challenge/stage_2_...
...,...,...
21674,fffb2395-8edd-4954-8a89-ffe2fd329be3,../rsna-pneumonia-detection-challenge/stage_2_...
7729,fffba05a-1635-4545-9bbd-57ad4cfe8d27,../rsna-pneumonia-detection-challenge/stage_2_...
6091,fffc95b5-605b-4226-80ab-62caec682b22,../rsna-pneumonia-detection-challenge/stage_2_...
14272,fffcff11-d018-4414-971a-a7cefa327795,../rsna-pneumonia-detection-challenge/stage_2_...


In [10]:
df_valid_data

,patientId,filepath
7634,0004cfab-14fd-4e49-80ba-63a80b6bddd6,../rsna-pneumonia-detection-challenge/stage_2_...
10664,000fe35a-2649-43d4-b027-e67796d412e0,../rsna-pneumonia-detection-challenge/stage_2_...
2520,0022073f-cec8-42ec-ab5f-bc2314649235,../rsna-pneumonia-detection-challenge/stage_2_...
2539,0025d2de-bd78-4d36-9f72-e15a5e22ca82,../rsna-pneumonia-detection-challenge/stage_2_...
17216,00293de0-a530-41dc-9621-0b3def01d06d,../rsna-pneumonia-detection-challenge/stage_2_...
...,...,...
19331,ffc01e64-ba14-4620-8016-235fc1609767,../rsna-pneumonia-detection-challenge/stage_2_...
9675,ffcc35e8-6fe8-42df-9605-9fedf57240d1,../rsna-pneumonia-detection-challenge/stage_2_...
24111,ffd56560-6754-4a20-83c6-06ad5c4b30ab,../rsna-pneumonia-detection-challenge/stage_2_...
14690,ffd787b6-59ca-48cb-bd15-bcedd52cf37c,../rsna-pneumonia-detection-challenge/stage_2_...


In [11]:
### Creating targets for model using Dataset Class

class mydataset(Dataset):

    def __init__(self, df_data, df_annot, mode = "train", transforms = None):
        
        super().__init__()
        self.pids = df_data['patientId'].unique()
        self.df_annot = df_annot
        self.df_data = df_data
        self.mode = mode
        self.transforms = transforms

    def __getitem__(self, idx):

        # Retriving image id and records from df
        pid = self.pids[idx]
        annot = self.df_annot[self.df_annot['patientId'] == pid]

        # Loading Image
        fp = self.df_data[self.df_data['patientId'] == pid]['filepath'].item()
        image = dicom.dcmread(fp)
        image = image.pixel_array.astype(np.float32)
        image = np.expand_dims(image, axis=0).transpose(1, 2, 0)
        
        # If mode is set to train, then only we create targets
        if self.mode == "train" or self.mode == "valid":
            
            if annot['x'].isnull().values.any():
                boxes = np.zeros((0,5))
                sample = {'img':image, 'annot':boxes}
            else:
                # Converting xmin, ymin, w, h to x1, y1, x2, y2
                boxes = np.zeros((annot.shape[0], 5))

                boxes[:, 0:4] = annot[['x', 'y', 'width', 'height']].values
                boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
                boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
                boxes[:, 4] = 1 # This is for label, as we have only 1 class, it is always 1

            sample = {'img': image, 'annot': boxes}
            if self.transforms:
                sample = self.transforms(sample)

            return sample
        
        elif self.mode == "test":
            
            sample = {'img' : image}
            if self.transforms:
                sample = self.transforms(sample)
                
            return sample
        

    def __len__(self):
        return self.pids.shape[0]
    

In [12]:
# train_dataset = mydataset(df_train_data, df_annot, mode="train", transforms=T.Compose([Resizer()]))
# valid_dataset = mydataset(df_valid_data, df_annot, mode= "valid", transforms = T.Compose([Resizer()]))
# torch.save(train_dataset, './train_128.dataset')
# torch.save(valid_dataset, './valid_128.dataset')
train_dataset = torch.load('./train_128.dataset')
valid_dataset = torch.load('./valid_128.dataset')
# train_dataset = mydataset(df_train_data, df_annot, mode="train")
# valid_dataset = mydataset(df_valid_data, df_annot, mode= "valid")

In [13]:
# cd = 5
# for each in train_dataset:
#     print('img:\n', each['img'].size(), '\n\t', each['img'])
#     print('annot:\n', each['annot'].size(), '\n\t', each['annot'])
#     print('scale:', each['scale'])
#     cd -= 1
#     if cd == 0:
#         break

In [14]:
# DataLoaders
train_dataloader = DataLoader(
    train_dataset,
    batch_size = 128,
    shuffle = True,
    collate_fn = collater
)

valid_dataloader = DataLoader(
    valid_dataset,
    batch_size = 128,
    shuffle = False,
    collate_fn = collater
)

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [16]:
for each in train_dataloader:
    img = each['img']
    annot = each['annot']
    break

/home/jovyan/work/final_proj/pytorch-retinanet/retinanet/dataloader.py:318: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_imgs[i, :int(img.shape[0]), :int(img.shape[1]), :] = torch.tensor(img)
/home/jovyan/work/final_proj/pytorch-retinanet/retinanet/dataloader.py:330: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  annot_padded[idx, :annot.shape[0], :] = torch.tensor(annot)


In [17]:
img.size(), annot.size()

(torch.Size([128, 1, 128, 128]), torch.Size([128, 2, 5]))

In [18]:
# def get_predict(retinanet, dataloader, confidence):

#     # retinanet = torch.load('./retinanet_resnet50.pt')
#     retinanet.eval()
#     scores_list, labels_list, boxes_list = [], [], []

#     with torch.no_grad():
        
#         for data in tqdm(dataloader):
#             img = data['img']
#             annot = data['annot']
#             # scale = data['scale']

#             # run network
#             if torch.cuda.is_available():
#                 scores, labels, boxes = retinanet(img.cuda().float())
#             else:
#                 scores, labels, boxes = retinanet(img.float())

#             scores_list.append(scores.cpu())
#             labels_list.append(labels.cpu())
#             boxes_list.append(boxes.cpu())

#             # print(scores, "\n", labels, "\n", boxes, "\n")

#             # correct boxes for image scale
#             # boxes /= scale
        
#         scores_list = torch.cat(scores_list, dim=0).numpy()
#         labels_list = torch.cat(labels_list, dim=0).numpy()
#         boxes_list = torch.cat(boxes_list, dim=0).numpy()
            
#         predict = {}
#         for score_threshold in confidence:
#             predict[score_threshold] = [[None, None] for j in range(len(valid_dataset))]

#             for idx in range(len(scores_list)):
#                 scores = scores_list[idx]
#                 labels = labels_list[idx]
#                 boxes = boxes_list[idx]

#                 indices = np.where(scores > score_threshold)[0]
#                 if indices.shape[0] > 0:
#                     # select those scores
#                     scores = scores[indices]

#                     # find the order with which to sort the scores
#                     scores_sort = np.argsort(-scores)[:4]

#                     # select detections
#                     image_boxes      = boxes[indices[scores_sort], :]
#                     image_scores     = scores[scores_sort]
#                     image_labels     = labels[indices[scores_sort]]
#                     image_detections = np.concatenate([image_boxes, np.expand_dims(image_scores, axis=1), np.expand_dims(image_labels, axis=1)], axis=1)

#                     # copy detections to predict
#                     for label in range(2):
#                         predict[score_threshold][idx][label] = image_detections[image_detections[:, -1] == label, :-1]
#                 else:
#                     # copy detections to predict
#                     for label in range(2):
#                         predict[score_threshold][idx][label] = np.zeros((0, 5))
                    
#     return predict
#             # print('{}/{}'.format(index + 1, len(dataset)), end='\r')
#             # break

In [19]:
def get_predict(retinanet, num, dataset, confidence):

    # retinanet = torch.load('./retinanet_resnet50.pt')
    retinanet.eval()
    scores_list, labels_list, boxes_list = [], [], []

    with torch.no_grad():
        
        for index in tqdm(range(num)):
            data = dataset[index]
            img = data['img'].permute(2, 0, 1).float()
            img = np.repeat(img, 3, axis=0)
            annot = data['annot']
            scale = data['scale']

            # run network
            if torch.cuda.is_available():
                scores, labels, boxes = retinanet(img.cuda().unsqueeze(dim=0))
            else:
                scores, labels, boxes = retinanet(img.unsqueeze(dim=0))

            scores_list.append(scores.cpu().numpy())
            labels_list.append(labels.cpu().numpy())
            boxes_list.append(boxes.cpu().numpy())

            # print(scores, "\n", labels, "\n", boxes, "\n")

            # correct boxes for image scale
            # boxes /= scale

            # select indices which have a score above the threshold
            
        predict = {}
        for score_threshold in confidence:
            predict[score_threshold] = [[None, None] for j in range(len(valid_dataset))]

            for idx in range(len(scores_list)):
                scores = scores_list[idx]
                labels = labels_list[idx]
                boxes = boxes_list[idx]

                indices = np.where(scores > score_threshold)[0]
                if indices.shape[0] > 0:
                    # select those scores
                    scores = scores[indices]

                    # find the order with which to sort the scores
                    scores_sort = np.argsort(-scores)[:4]

                    # select detections
                    image_boxes      = boxes[indices[scores_sort], :]
                    image_scores     = scores[scores_sort]
                    image_labels     = labels[indices[scores_sort]]
                    image_detections = np.concatenate([image_boxes, np.expand_dims(image_scores, axis=1), np.expand_dims(image_labels, axis=1)], axis=1)

                    # copy detections to predict
                    for label in range(2):
                        predict[score_threshold][idx][label] = image_detections[image_detections[:, -1] == label, :-1]
                else:
                    # copy detections to predict
                    for label in range(2):
                        predict[score_threshold][idx][label] = np.zeros((0, 5))
                    
    return predict
            # print('{}/{}'.format(index + 1, len(dataset)), end='\r')
            # break

In [20]:
retinanet = model.resnet34(num_classes = 2, pretrained = True)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to ./resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [21]:
optimizer = torch.optim.Adam(retinanet.parameters(), lr = 0.0001)
retinanet.to(device)
epochs = 15

In [22]:
def train_one_epoch(retinanet, epoch_num, train_data_loader):
    
    print("Epoch - {} Started".format(epoch_num))
    st = time.time()
    
    retinanet.train()
    
    epoch_loss = []

    for iter_num, data in enumerate(train_data_loader):

        # Reseting gradients after each iter
        optimizer.zero_grad()

        # Forward
        img = data['img']
        img = np.repeat(img, 3, axis=1)
        classification_loss, regression_loss = retinanet([img.cuda().float(), data['annot'].cuda().float()])

        # Calculating Loss
        classification_loss = classification_loss.mean()
        regression_loss = regression_loss.mean()

        loss = classification_loss + regression_loss

        if bool(loss == 0):
            continue

        # Calculating Gradients
        loss.backward()

        # Gradient Clipping
        torch.nn.utils.clip_grad_norm_(retinanet.parameters(), 0.1)
                
        # Updating Weights
        optimizer.step()

        #Epoch Loss
        epoch_loss.append(float(loss))


        print(
            'Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Running loss: {:1.5f}'.format(
                epoch_num, iter_num, float(classification_loss), float(regression_loss), np.mean(epoch_loss)))

        del classification_loss
        del regression_loss
        
    # Update the learning rate
    #if lr_scheduler is not None:
        #lr_scheduler.step()
    
    et = time.time()
    print("\n Total Time - {}\n".format(int(et - st)))
    torch.save(retinanet, "retinanet_resnet34_pretrained.pt")
    
    ## validation
    # retinanet = torch.load('./retinanet_resnet34.pt')
    retinanet.eval()
    with torch.no_grad():

        results = []
        scores_ = None; labels_=None; boxes_=None

        print("get detections on train dataset...")
        train_detection_dict = get_predict(retinanet, 2000, train_dataset, confidence=[.05, .1, .15, .2, .25, .3, .35, .4, .45, .5])
        print("get detections on valid dataset...")
        val_detection_dict = get_predict(retinanet, len(valid_dataset), valid_dataset, confidence=[.05, .1, .15, .2, .25, .3, .35, .4, .45, .5])

        cd = 0; sz = 0
        for each in [train_detection_dict, val_detection_dict]:

            print()
            print("="*100)
            if cd == 0:
                print("mAP on training dataset")
                sz = 2000
            else:
                print("mAP on validating dataset")
                sz = len(valid_dataset)
            print("="*100)

            for score_threshold in each:

                print("\npredicting with confidence value: {}".format(score_threshold))
                all_detections = each[score_threshold]
                if cd == 0:
                    all_annotations = torch.load('./ground_truth_annot_train.data')
                else:
                    all_annotations = torch.load('./ground_truth_annot_valid.data')

                # print(all_annotations)

                tmp = []

                for iou_threshold in {0.3, 0.4, 0.5, 0.6, 0.7}:
                    # print("in")

                    average_precisions = {}

                    for label in range(1,2):
                        # print("in label")
                        false_positives = np.zeros((0,))
                        true_positives  = np.zeros((0,))
                        scores          = np.zeros((0,))
                        num_annotations = 0.0

                        for i in range(sz):
                            detections           = all_detections[i][label]
                            annotations          = all_annotations[i][label]
                            num_annotations     += annotations.shape[0]
                            detected_annotations = []

                            # print(detections)

                            for d in detections:
                                scores = np.append(scores, d[4])

                                if annotations.shape[0] == 0:
                                    false_positives = np.append(false_positives, 1)
                                    true_positives  = np.append(true_positives, 0)
                                    continue

                                overlaps            = compute_overlap(np.expand_dims(d, axis=0), annotations)
                                assigned_annotation = np.argmax(overlaps, axis=1)
                                max_overlap         = overlaps[0, assigned_annotation]

                                # print(assigned_annotation)
                                # print(overlaps)
                                # print(max_overlap)
                                # print()

                                # print(overlaps)

                                if max_overlap >= iou_threshold and assigned_annotation not in detected_annotations:
                                    false_positives = np.append(false_positives, 0)
                                    true_positives  = np.append(true_positives, 1)
                                    detected_annotations.append(assigned_annotation)
                                else:
                                    false_positives = np.append(false_positives, 1)
                                    true_positives  = np.append(true_positives, 0)

                        # print(false_positives)
                        # print(true_positives)

                        # print(num_annotations)

                        # no annotations -> AP for this class is 0 (is this correct?)
                        if num_annotations == 0:
                            average_precisions[label] = 0, 0
                            continue

                        # sort by score
                        indices         = np.argsort(-scores)
                        false_positives = false_positives[indices]
                        true_positives  = true_positives[indices]

                        # compute false positives and true positives
                        false_positives = np.cumsum(false_positives)
                        true_positives  = np.cumsum(true_positives)

                        # compute recall and precision
                        recall    = true_positives / num_annotations
                        precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)

                        # compute average precision
                        average_precision  = _compute_ap(recall, precision)
                        average_precisions[label] = average_precision, num_annotations

                    # print(precision.shape[0])
                    if precision.shape[0] == 0:
                        print("AP@{}:0.0000,Precision:0.0000,Recall:0.0000".format(iou_threshold))
                        continue

                    fmt = "AP@{}:{:.4f},Precision:{:.4f},Recall:{:.4f}"
                    print(fmt.format(iou_threshold, average_precisions[1][0], precision[-1], recall[-1]))
                
                cd += 1



In [ ]:
### Training Loop
for epoch in range(epochs):
    train_one_epoch(retinanet, epoch, train_dataloader)

Epoch - 0 Started
Epoch: 0 | Iteration: 0 | Classification loss: 0.24174 | Regression loss: 0.21143 | Running loss: 0.45317
Epoch: 0 | Iteration: 1 | Classification loss: 0.17955 | Regression loss: 0.16965 | Running loss: 0.40119
Epoch: 0 | Iteration: 2 | Classification loss: 0.22877 | Regression loss: 0.17179 | Running loss: 0.40098
Epoch: 0 | Iteration: 3 | Classification loss: 0.17339 | Regression loss: 0.16184 | Running loss: 0.38454
Epoch: 0 | Iteration: 4 | Classification loss: 0.13365 | Regression loss: 0.13851 | Running loss: 0.36207
Epoch: 0 | Iteration: 5 | Classification loss: 0.11614 | Regression loss: 0.10427 | Running loss: 0.33846
Epoch: 0 | Iteration: 6 | Classification loss: 0.20561 | Regression loss: 0.20520 | Running loss: 0.34879
Epoch: 0 | Iteration: 7 | Classification loss: 0.16230 | Regression loss: 0.15903 | Running loss: 0.34536
Epoch: 0 | Iteration: 8 | Classification loss: 0.15967 | Regression loss: 0.15447 | Running loss: 0.34189
Epoch: 0 | Iteration: 9 | Cl

  0%|          | 0/2000 [00:00<?, ?it/s]

get detections on valid dataset...


  0%|          | 0/5435 [00:00<?, ?it/s]


mAP on training dataset

predicting with confidence value: 0.05
AP@0.3:0.4696,Precision:0.1247,Recall:0.8076
AP@0.4:0.3385,Precision:0.1037,Recall:0.6716
AP@0.6:0.0855,Precision:0.0580,Recall:0.3756
AP@0.5:0.1941,Precision:0.0836,Recall:0.5412
AP@0.7:0.0150,Precision:0.0241,Recall:0.1563

predicting with confidence value: 0.1
AP@0.3:0.0106,Precision:0.0427,Recall:0.1695
AP@0.4:0.0056,Precision:0.0303,Recall:0.1199
AP@0.6:0.0004,Precision:0.0069,Recall:0.0274
AP@0.5:0.0014,Precision:0.0151,Recall:0.0600
AP@0.7:0.0000,Precision:0.0020,Recall:0.0078

predicting with confidence value: 0.15
AP@0.3:0.0073,Precision:0.0487,Recall:0.0965
AP@0.4:0.0038,Precision:0.0336,Recall:0.0665
AP@0.6:0.0003,Precision:0.0086,Recall:0.0169
AP@0.5:0.0010,Precision:0.0171,Recall:0.0339
AP@0.7:0.0000,Precision:0.0026,Recall:0.0052

predicting with confidence value: 0.2
AP@0.3:0.0054,Precision:0.0577,Recall:0.0626
AP@0.4:0.0031,Precision:0.0433,Recall:0.0469
AP@0.6:0.0002,Precision:0.0096,Recall:0.0104
AP@0.5:

...


mAP on training dataset

predicting with confidence value: 0.05
AP@0.3:0.6014,Precision:0.3182,Recall:0.8474
AP@0.4:0.5170,Precision:0.2876,Recall:0.7660
AP@0.6:0.2355,Precision:0.2008,Recall:0.5347
AP@0.5:0.4085,Precision:0.2511,Recall:0.6688
AP@0.7:0.0719,Precision:0.1101,Recall:0.2932

predicting with confidence value: 0.1
AP@0.3:0.0066,Precision:0.0488,Recall:0.0939
AP@0.4:0.0030,Precision:0.0312,Recall:0.0600
AP@0.6:0.0003,Precision:0.0095,Recall:0.0183
AP@0.5:0.0010,Precision:0.0156,Recall:0.0300
AP@0.7:0.0001,Precision:0.0034,Recall:0.0065

predicting with confidence value: 0.15
AP@0.3:0.0051,Precision:0.0559,Recall:0.0652
AP@0.4:0.0024,Precision:0.0358,Recall:0.0417
AP@0.6:0.0002,Precision:0.0101,Recall:0.0117
AP@0.5:0.0009,Precision:0.0190,Recall:0.0222
AP@0.7:0.0001,Precision:0.0034,Recall:0.0039

predicting with confidence value: 0.2
AP@0.3:0.0040,Precision:0.0627,Recall:0.0456
AP@0.4:0.0019,Precision:0.0412,Recall:0.0300
AP@0.6:0.0002,Precision:0.0125,Recall:0.0091
AP@0.5:

  0%|          | 0/2000 [00:00<?, ?it/s]

get detections on valid dataset...


  0%|          | 0/5435 [00:00<?, ?it/s]


mAP on training dataset

predicting with confidence value: 0.05
AP@0.3:0.7197,Precision:0.3163,Recall:0.8529
AP@0.4:0.6531,Precision:0.2943,Recall:0.7937
AP@0.6:0.3843,Precision:0.2244,Recall:0.6050
AP@0.5:0.5463,Precision:0.2652,Recall:0.7151
AP@0.7:0.1687,Precision:0.1448,Recall:0.3904

predicting with confidence value: 0.1
AP@0.3:0.0100,Precision:0.0488,Recall:0.0978
AP@0.4:0.0020,Precision:0.0260,Recall:0.0522
AP@0.6:0.0001,Precision:0.0052,Recall:0.0104
AP@0.5:0.0004,Precision:0.0124,Recall:0.0248
AP@0.7:0.0000,Precision:0.0020,Recall:0.0039

predicting with confidence value: 0.15
AP@0.3:0.0082,Precision:0.0547,Recall:0.0665
AP@0.4:0.0013,Precision:0.0258,Recall:0.0313
AP@0.6:0.0001,Precision:0.0054,Recall:0.0065
AP@0.5:0.0003,Precision:0.0118,Recall:0.0143
AP@0.7:0.0000,Precision:0.0011,Recall:0.0013

predicting with confidence value: 0.2
AP@0.3:0.0075,Precision:0.0663,Recall:0.0535
AP@0.4:0.0012,Precision:0.0307,Recall:0.0248
AP@0.6:0.0000,Precision:0.0049,Recall:0.0039
AP@0.5:

### Show some images

In [ ]:
# from matplotlib.patches import Rectangle
# retinanet = torch.load('./retinanet_resnet34.pt')val_detection_dict

In [ ]:
# ### Sample Results
# # retinanet = model.resnet50(num_classes = 2, pretrained = False)

# retinanet.eval()
# with torch.no_grad():
# # unnormalize = UnNormalizer()

#     fig, ax = plt.subplots(2, 8, figsize=(32, 8))

#     for iter_num, data in enumerate(valid_dataloader):

#         # print(data['annot'].size())

#         tmp = data['img'].to(device)

#         # Getting Predictions
#         scores, classification, transformed_anchors = retinanet(tmp.float())
#         # print(scores)

#         idxs = np.where(scores.cpu()>0.05)
#         # print(scores)
#         # print(classification)
#         # print(transformed_anchors)
#         # print(idxs)
#         # img = np.array(255 * unnormalize(data['img'][0, :, :, :])).copy()

#         img = np.array(data['img'][0, :, :, :]).copy()
#         # img[img<0] = 0
#         # img[img>255] = 255
#         img = img.squeeze(0)
#         # print(img.shape)

#         ax[0][iter_num].imshow(img, cmap='gray')
#         ax[1][iter_num].imshow(img, cmap='gray')

#         for j in range(idxs[0].shape[0]):
#             ## predict
#             bbox = transformed_anchors[idxs[0][j], :]
#             x1 = int(bbox[0])
#             y1 = int(bbox[1])
#             x2 = int(bbox[2])
#             y2 = int(bbox[3])
#             rect = Rectangle((x1,y1),x2-x1,y2-y1,linewidth=1,edgecolor='r',facecolor='none')
#             ax[0][iter_num].add_patch(rect)
            
#         for j in range(data['annot'].size(1)):
#             ## ground truth
#             bbox = data['annot'][0][j]
#             x1 = int(bbox[0])
#             y1 = int(bbox[1])
#             x2 = int(bbox[2])
#             y2 = int(bbox[3])
#             rect = Rectangle((x1,y1),x2-x1,y2-y1,linewidth=1,edgecolor='r',facecolor='none')
#             ax[1][iter_num].add_patch(rect)

#         if iter_num == 7:
#             break



#     plt.show()


In [ ]:
# score_threshold = 0.1

# predict = [[None, None] for j in range(len(valid_dataset))]

# retinanet = torch.load('./retinanet_gwd.pt')
# retinanet.eval()

# with torch.no_grad():

#     for index in tqdm(range(len(valid_dataset))):
#         data = valid_dataset[index]
#         img = data['img']
#         annot = data['annot']
#         scale = data['scale']

#         # run network
#         if torch.cuda.is_available():
#             scores, labels, boxes = retinanet(img.cuda().permute(2, 0, 1).float().unsqueeze(dim=0))
#         else:
#             scores, labels, boxes = retinanet(data['img'].permute(2, 0, 1).float().unsqueeze(dim=0))
            
#         scores = scores.cpu().numpy()
#         labels = labels.cpu().numpy()
#         boxes  = boxes.cpu().numpy()
        
#         # print(scores, "\n", labels, "\n", boxes, "\n")

#         # correct boxes for image scale
#         # boxes /= scale

#         # select indices which have a score above the threshold
#         indices = np.where(scores > score_threshold)[0]
#         if indices.shape[0] > 0:
#             # select those scores
#             scores = scores[indices]

#             # find the order with which to sort the scores
#             scores_sort = np.argsort(-scores)[:4]

#             # select detections
#             image_boxes      = boxes[indices[scores_sort], :]
#             image_scores     = scores[scores_sort]
#             image_labels     = labels[indices[scores_sort]]
#             image_detections = np.concatenate([image_boxes, np.expand_dims(image_scores, axis=1), np.expand_dims(image_labels, axis=1)], axis=1)

#             # copy detections to predict
#             for label in range(2):
#                 predict[index][label] = image_detections[image_detections[:, -1] == label, :-1]
#         else:
#             # copy detections to predict
#             for label in range(2):
#                 predict[index][label] = np.zeros((0, 5))

#         # print('{}/{}'.format(index + 1, len(dataset)), end='\r')
#         # break

In [ ]:
# ground_truth = [[None, None] for _ in range(2000)]
# for index in tqdm(range(2000)):
#     data = train_dataset[index]
#     annot = data['annot'].numpy()
#     if annot.shape[0] != 0:
#         for label in range(2):
#             ground_truth[index][label] = annot[annot[:, -1] == label, :-1]
#     else:
#         for label in range(2):
#             ground_truth[index][label] = np.zeros((0, 5))

In [ ]:
# torch.save(ground_truth, "./ground_truth_annot_train.data")

In [ ]:
# ground_truth = [[None, None] for _ in range(len(valid_dataset))]
# for index in tqdm(range(len(valid_dataset))):
#     data =valid_dataset[index]
#     annot = data['annot'].numpy()
#     if annot.shape[0] != 0:
#         for label in range(2):
#             ground_truth[index][label] = annot[annot[:, -1] == label, :-1]
#     else:
#         for label in range(2):
#             ground_truth[index][label] = np.zeros((0, 5))

In [ ]:
# torch.save(ground_truth, "./ground_truth_annot_valid.data")